<a href="https://colab.research.google.com/github/ushasri999/Multiclass-classification-of-DDoS-attacks-in-IoT-network-using-hybrid-feature-selection-algorithm/blob/main/XGB12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**12 classes, 10 features**

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Step 1: Load and filter the dataset
df = pd.read_csv('/content/drive/MyDrive/major project/data.csv')  # Update path as needed
df_ddos = df[df['label'].str.contains('DDoS', case=False, na=False)]  # Filter DDoS labels

# Step 2: Separate features and target
X = df_ddos.drop("label", axis=1, errors='ignore')  # Drop target column
y = df_ddos["label"] if 'label' in df.columns else pd.Series([])  # Extract target column

# Encode target labels into numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Check for valid features and labels
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Step 3: Handle categorical and missing data
X = pd.get_dummies(X, drop_first=True)  # One-hot encode categorical variables
X.fillna(X.mean(), inplace=True)  # Fill missing values with column means

Features shape: (173777, 46)
Target shape: (173777,)


In [ ]:
# Step 4: Hybrid Feature Selection
# Filter Method: SelectKBest
filter_selector = SelectKBest(score_func=f_classif, k=5)
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with XGBoost
rfe_selector = RFE(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                   n_features_to_select=5)
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 10 features from both methods
top_features_filter = np.argsort(filter_scores)[-5:]  # Top 10 from SelectKBest
print('top_features_filter ', top_features_filter)
top_features_rfe = np.where(rfe_ranking == 1)[0]       # Top 10 from RFE
print('top_features_rfe ', top_features_rfe)
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Union
print('top_features_combined ', top_features_combined)

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]
print('X_selected ', X_selected)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 24 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:56:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:56:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:57:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

top_features_filter  [24 23 13 12 22]
top_features_rfe  [ 7  8 10 27 39]
top_features_combined  [39, 8, 7, 10, 12, 13, 22, 23, 24, 27]
X_selected                   IAT  syn_flag_number  fin_flag_number  psh_flag_number  \
0       8.334383e+07              0.0              1.0              0.0   
2       8.312799e+07              0.0              0.0              0.0   
6       8.336548e+07              1.0              0.0              0.0   
9       8.308906e+07              1.0              0.0              0.0   
10      8.333087e+07              0.0              0.0              1.0   
...              ...              ...              ...              ...   
238682  8.334449e+07              0.0              1.0              0.0   
238683  8.331392e+07              0.0              0.0              1.0   
238684  8.308883e+07              1.0              0.0              0.0   
238685  8.333177e+07              0.0              0.0              1.0   
238686  8.312453e+07        

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = xgb_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:24:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9965857214102122
Precision: 0.9960949338079919
Recall: 0.9965857214102122
F1 Score: 0.996089794674775
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       441
           1       0.88      0.15      0.25        48
           2       0.99      1.00      1.00     10925
           3       0.96      0.92      0.94       726
           4       1.00      1.00      1.00      6409
           5       1.00      1.00      1.00      6131
           6       1.00      1.00      1.00      6173
           7       0.00      0.00      0.00        21
           8       1.00      1.00      1.00      5436
           9       1.00      1.00      1.00      6982
          10       1.00      1.00      1.00      8369
          11       0.93      0.96      0.95       473

    accuracy                           1.00     52134
   macro avg       0.90      0.83      0.84     52134
weighted avg       1.00      1.00      1.00     52

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

**DDoS attacks only, 15 features**

In [ ]:
# Step 4: Hybrid Feature Selection
# Filter Method: SelectKBest
filter_selector = SelectKBest(score_func=f_classif, k=8)
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with XGBoost
rfe_selector = RFE(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                   n_features_to_select=7)
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 10 features from both methods
top_features_filter = np.argsort(filter_scores)[-8:]  # Top 10 from SelectKBest
print('top_features_filter ', top_features_filter)
top_features_rfe = np.where(rfe_ranking == 1)[0]       # Top 10 from RFE
print('top_features_rfe ', top_features_rfe)
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Union
print('top_features_combined ', top_features_combined)

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]
print('X_selected ', X_selected)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 24 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:37:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:37:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:38:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

top_features_filter  [ 7 28 25 24 23 13 12 22]
top_features_rfe  [ 7  8 10 27 30 34 39]
top_features_combined  [34, 7, 8, 39, 10, 12, 13, 22, 23, 24, 25, 27, 28, 30]
X_selected           Min  fin_flag_number  syn_flag_number           IAT  psh_flag_number  \
0       54.0              1.0              0.0  8.334383e+07              0.0   
2       42.0              0.0              0.0  8.312799e+07              0.0   
6       54.0              0.0              1.0  8.336548e+07              0.0   
9       54.0              0.0              1.0  8.308906e+07              0.0   
10      54.0              0.0              0.0  8.333087e+07              1.0   
...      ...              ...              ...           ...              ...   
238682  54.0              1.0              0.0  8.334449e+07              0.0   
238683  54.0              0.0              0.0  8.331392e+07              1.0   
238684  54.0              0.0              1.0  8.308883e+07              0.0   
238685  54.0

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = xgb_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:53:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.999156021022749
Precision: 0.9991765233746613
Recall: 0.999156021022749
F1 Score: 0.9991595976946213
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       441
           1       0.90      0.77      0.83        48
           2       1.00      1.00      1.00     10925
           3       1.00      1.00      1.00       726
           4       1.00      1.00      1.00      6409
           5       1.00      1.00      1.00      6131
           6       1.00      1.00      1.00      6173
           7       0.56      0.67      0.61        21
           8       1.00      1.00      1.00      5436
           9       1.00      1.00      1.00      6982
          10       1.00      1.00      1.00      8369
          11       0.98      0.98      0.98       473

    accuracy                           1.00     52134
   macro avg       0.95      0.95      0.95     52134
weighted avg       1.00      1.00      1.00     521

**DDoS attacks only, 5 features**

In [ ]:
# Step 4: Hybrid Feature Selection
# Filter Method: SelectKBest
filter_selector = SelectKBest(score_func=f_classif, k=3)  # Select top 5 features
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with XGBoost
rfe_selector = RFE(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                   n_features_to_select=2)  # Select top 5 features
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 15 features from both methods
top_features_filter = np.argsort(filter_scores)[-3:]  # Top 5 from SelectKBest
top_features_rfe = np.where(rfe_ranking == 1)[0]       # Top 5 from RFE
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Union

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 24 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:57:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:57:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:58:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = xgb_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [16:14:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.4476157593892661
Precision: 0.29816814669421704
Recall: 0.4476157593892661
F1 Score: 0.3296899240980541
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       441
           1       0.00      0.00      0.00        48
           2       0.28      1.00      0.43     10925
           3       0.00      0.00      0.00       726
           4       1.00      0.98      0.99      6409
           5       1.00      1.00      1.00      6131
           6       0.00      0.00      0.00      6173
           7       0.00      0.00      0.00        21
           8       0.00      0.00      0.00      5436
           9       0.00      0.00      0.00      6982
          10       0.00      0.00      0.00      8369
          11       0.00      0.00      0.00       473

    accuracy                           0.45     52134
   macro avg       0.19      0.25      0.20     52134
weighted avg       0.30      0.45      0.33     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m